In [1]:
import pandas as pd
import datasets
import os

In [2]:
# data_files = {"train":"train.parquet",
#               "test":"test.parquet"
#              }
df = datasets.load_dataset('AfricaKing/TSSB-3M',
                     # data_files=data_files
                           split='train[:5%]'
                          )

Resolving data files:   0%|          | 0/36 [00:00<?, ?it/s]

In [3]:
# source feature
len(df['before'])

172062

In [4]:
# Create a split for the dataset
df

Dataset({
    features: ['project', 'commit_sha', 'parent_sha', 'file_path', 'project_url', 'likely_bug', 'comodified', 'in_function', 'diff', 'before', 'after', 'sstub_pattern', 'edit_script'],
    num_rows: 172062
})

In [5]:
# Scripting in Python functio n
def scriptFunction():
    return """
    #!/bin/bash \
    #ech0 'this"
    """

In [6]:
scriptFunction()

'\n    #!/bin/bash     #ech0 \'this"\n    '